In [83]:
from env import host, user, password
from datetime import datetime
import pandas as pd
import numpy as np
from pydataset import data

## Exercises I

In [3]:
def get_db_url(user, hostname, password, database_name):
    return f'mysql+pymysql://{user}:{password}@{hostname}/{database_name}'

In [13]:
url = get_db_url(user, host, password, 'employees')

In [8]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [9]:
url = url + '332'

In [11]:
# Intentionally make typo in database url
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'innis_1659'@'%' to database 'employees332'")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [12]:
# The url typo results in an OperationalError and Access denied

In [14]:
# Intentionally making error in SQL query
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFFSET 50', url)

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'OFFFSET 50' at line 1")
[SQL: SELECT * FROM employees LIMIT 5 OFFFSET 50]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [15]:
# That typo in the SQL query resulted in a Programming Error

In [16]:
employees = pd.read_sql('SELECT * FROM employees', url)
titles = pd.read_sql('SELECT * FROM titles', url)

7. How many rows and columns for each DataFrame

In [18]:
# Rows and columns for each dataframe matches SQL database
employees.shape 

(300024, 6)

In [19]:
titles.shape

(443308, 4)

In [20]:
# Row and column numbers match the SQL database (checked using Sequel-Ace)

8. Summary statistics for each dataframe

In [28]:
employees.describe(include = 'all')

,emp_no,birth_date,first_name,last_name,gender,hire_date
count,300024.000000,300024,300024,300024,300024,300024
unique,NaN,4750,1275,1637,2,5434
top,NaN,1952-03-08,Shahab,Baba,M,1985-06-20
freq,NaN,95,295,226,179973,132
mean,253321.763392,NaN,NaN,NaN,NaN,NaN
std,161828.235540,NaN,NaN,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN,NaN,NaN
25%,85006.750000,NaN,NaN,NaN,NaN,NaN
50%,249987.500000,NaN,NaN,NaN,NaN,NaN
75%,424993.250000,NaN,NaN,NaN,NaN,NaN


In [29]:
titles.describe(include = 'all')

,emp_no,title,from_date,to_date
count,443308.000000,443308,443308,443308
unique,NaN,7,6393,5888
top,NaN,Engineer,1998-10-25,9999-01-01
freq,NaN,115003,132,240124
mean,253075.034430,NaN,NaN,NaN
std,161853.292613,NaN,NaN,NaN
min,10001.000000,NaN,NaN,NaN
25%,84855.750000,NaN,NaN,NaN
50%,249847.500000,NaN,NaN,NaN
75%,424891.250000,NaN,NaN,NaN


9. How many unique titles in titles DataFrame

In [35]:
# We can see 7 unique titles as show above using describe(). Also can generate list of the titles and number as follows:

print(f"There are {len(titles.title.unique())} unique title(s) in the titles DataFrame")

There are 7 unique title(s) in the titles DataFrame


10. Oldest date in the to_date column

In [38]:
print(f"The oldest date in the to_date column for the titles table is {titles.to_date.min()}")

The oldest date in the to_date column for the titles table is 1985-03-01


11. Most recent date in the to_date column

In [66]:
print(f"The most recent date in the to_date column for the titles table (besides {titles.to_date.max()}) is {titles.to_date[titles.to_date <=datetime.today().date()].max()}")

The most recent date in the to_date column for the titles table (besides 9999-01-01) is 2002-08-01


## Exercises II

1. Copy the users and roles DataFrames from the examples above.

In [69]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [70]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


What is the result of using a right join on the DataFrames?

In [72]:
users.merge(roles, how = "right")

,id,name,role_id
0,1,admin,NaN
1,2,author,NaN
2,3,reviewer,NaN
3,4,commenter,NaN


What is the result of using an outer join on the DataFrames?

In [73]:
users.merge(roles, how = "outer")

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN
6,1,admin,NaN
7,2,author,NaN
8,3,reviewer,NaN
9,4,commenter,NaN


What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [81]:
users.drop(columns='role_id').merge(roles.drop(columns = 'id'))

,id,name


Load the mpg dataset from PyDataset.

In [109]:
mpg = data('mpg')

Output and read the documentation for the mpg dataset.

In [90]:
data('mpg',show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




How many rows and columns are in the dataset?

In [96]:
print(f'There are {mpg.shape[0]} rows and {mpg.shape[1]} columns in the mpg dataset')

There are 234 rows and 11 columns in the mpg dataset


Check out your column names and perform any cleanup you may want on them.

In [110]:
mpg = mpg.rename(columns = {'cty':'city_mpg','hwy':'highway_mpg','drv':'drive'})

In [114]:
mpg.describe(include='all')

,manufacturer,model,displ,year,cyl,trans,drive,city_mpg,highway_mpg,fl,class
count,234,234,234.000000,234.000000,234.000000,234,234,234.000000,234.000000,234,234
unique,15,38,NaN,NaN,NaN,10,3,NaN,NaN,5,7
top,dodge,caravan 2wd,NaN,NaN,NaN,auto(l4),f,NaN,NaN,r,suv
freq,37,11,NaN,NaN,NaN,83,106,NaN,NaN,168,62
mean,NaN,NaN,3.471795,2003.500000,5.888889,NaN,NaN,16.858974,23.440171,NaN,NaN
std,NaN,NaN,1.291959,4.509646,1.611534,NaN,NaN,4.255946,5.954643,NaN,NaN
min,NaN,NaN,1.600000,1999.000000,4.000000,NaN,NaN,9.000000,12.000000,NaN,NaN
25%,NaN,NaN,2.400000,1999.000000,4.000000,NaN,NaN,14.000000,18.000000,NaN,NaN
50%,NaN,NaN,3.300000,2003.500000,6.000000,NaN,NaN,17.000000,24.000000,NaN,NaN
75%,NaN,NaN,4.600000,2008.000000,8.000000,NaN,NaN,19.000000,27.000000,NaN,NaN


How many different manufacturers are there?

In [115]:
print(f'There are {len(mpg.manufacturer.unique())} unique manufacturers in this dataset')

There are 15 unique manufacturers in this dataset


How many different models are there?

In [117]:
print(f'There are {len(mpg.model.unique())} unique models in this dataset')

There are 38 unique models in this dataset


Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [118]:
mpg['mileage_difference'] = mpg.highway_mpg - mpg.city_mpg

Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [123]:
mpg['average_mileage'] = mpg[['highway_mpg','city_mpg']].mean(axis = 1)

Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

Do automatic or manual cars have better miles per gallon?